<a href="https://colab.research.google.com/github/iris-baptista/daa_seamCarving/blob/main/G20_111812_EurisaPatricio_122701_IrisBaptista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Minitrabalho DAA - "*Seam Carving*"
# Ano Letivo 2024/2025

## 1. Bibliotecas Importadas

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

## 2. Representação de Dados

### 2.1 Como está a representar a energia dos pixels no seu grafo? Qual foi o critério para esta escolha? Que tipo de grafo representa o problema em questão?

Optou-se por representar a energia dos pixels como peso das arestas no grafo. Os nós/vértices estarão associados aos pixels da imagem utilizada, e as arestas corresponderão à energia do pixel sucessor/destino, calculada com base na combinação linear do valor da intensidade dos pixels na sua vizinhança,conforme mencionado no enunciado, que neste caso, serão os pixels vizinhos no nível inferior. Contudo, sabe-se que o grafo será direconado, sendo assim um digrafo, pesado e acíclico, ou seja um DAG, **Nota** **(depois removo)( o que facilitará a encontrar um caminho mais curto, ou seja, de menos energia para que mais tarde possámos o remover)**

### 2.2 Qual é a representação computacional de grafo que está a utilizar? Por exemplo, matriz de adjacência, lista/mapa de adjacências ou uma outra alternativa?

A representação computacional do grafo que iremos utilizar é o **Mapa de Adjacências**, um dicionário onde cada pixel (representado por um objeto Vertex) terá como valor outro dicionário que apontará para os seus vizinhos inferiores, sendo o respetivo valor de cada vizinho será uma aresta (Edge) que contém a sua respetiva energia como peso.

### 2.3 Identifique as vantagens e desvantagens da sua representação de grafo escolhida e os critérios utilizados para a sua escolha. Por exemplo, a sua escolha facilita a implementação de alguma operação específica? Ou faz com que as operações fiquem mais eficientes (em relação ao tempo e ao espaço em memória)?

O mapa de adjacências

### 2.4 Se optou por utilizar as classes Graph/Digraph fornecidas na Semana 6 para a sua solução, identifique também as possíveis modificações que teve de realizar nas classes.

Responder...

## 3. API Seam Carving

### 3.1 Estrutura de Dados

In [ ]:
class Vertex:
    def __init__(self, coord):
        self._vertex_id = coord
        self._edges = []

    def __hash__(self):
        return hash(self._vertex_id)

    def __eq__(self, other):
        return self._vertex_id == other._vertex_id

    def vertex_id(self):
        return self._vertex_id

    def add_edge(self, vertex, weight):
        self._edges[vertex] = weight

    def get_edges(self):
        return self._edges

class Edge:
    def __init__(self, vertex_1, vertex_2, weight):
        self._vertex_1 = vertex_1
        self._vertex_2 = vertex_2
        self._weight = weight

    def cost(self):
        return self._weight

    def endpoints(self):
        return (self._vertex_1, self._vertex_2)

    def get_vertex_1(self):
        return self._vertex_1

    def get_vertex_2(self):
        return self._vertex_2


class Digraph:
    def __init__(self):
        super().__init__()
        self._in_adjancencies = {}     # dicionário com chave vértice e valor mapa de adjacências das arestas de entrada

    def add_edge(self, from_coord, to_coord, weight):
        if from_coord not in self._vertices:
            self.insert_vertex(from_coord)
        if to_coord not in self._vertices:
            self.insert_vertex(to_coord)

        vertex_u = self._vertices[from_coord]
        vertex_v = self._vertices[to_coord]

        if not self.has_edge(from_coord, to_coord):
            self._m += 1

        edge = Edge(vertex_u, vertex_v, weight)
        self._adjacencies[vertex_u][vertex_v] = edge
        vertex_u.add_edge(edge)

    def insert_vertex(self, coord):
        if coord not in self._vertices:
            vertex = Vertex(coord)
            self._vertices[coord] = vertex
            self._adjacencies[vertex] = {}
            self._n += 1

    def has_vertex(self, coord):
        return coord in self._vertices

    def has_edge(self, u_coord, v_coord):
        if not self.has_vertex(u_coord) or not self.has_vertex(v_coord):
            return False
        vertex_u = self._vertices[u_coord]
        vertex_v = self._vertices[v_coord]
        return vertex_v in self._adjacencies[vertex_u]

    def get_vertex(self, coord):
        return self._vertices.get(coord, None)



### 3.2 Calcular a energia da imagem

3.2.1 Implemente e teste a função auxiliar _calculate_energy() que irá calcular a energia da imagem atual. A função deve devolver o mapa da energia da imagem como um ndarray (numpy array).

3.2.2 Apresente uma análise da complexidade desta função em relação ao tempo e ao espaço extra de memória utilizados (em função da dimensão da imagem).

### 3.3 Encontrar a costura de menor energia (steam).

3.3.1 Descreva os passos e as modificações realizadas no grafo e/ou no algoritmo para reduzir o problema do seam carving para o problema do caminho mais curto num grafo (shortest path). Indique qual algoritmo shortest path que escolheu implementar e o motivo da sua escolha.

Resposta...

3.3.2 Implemente e teste o método find_vertical_steam()que deverá encontrar o caminho de menor energia.

3.3.3 Apresente uma análise completa da complexidade do seu algoritmo em função da dimensão da imagem, e compare-a com a complexidade de outras versões do algoritmo shortest path.

3.3.4 Analise e compare a complexidade do seu algoritmo baseado em grafos com uma solução que utiliza a estratégia de programação dinâmica. Na sua pesquisa bibliográfica e análise, deve usar fontes credíveis, tais como, artigos científicos, manuais técnicos ou recursos webgráficos devidamente acreditados.

### 3.4 Remover uma costura (steam) da imagem.

3.4.1 Implemente e teste o método remove_vertical_steam(steam) que recebe uma coleção com a sequência de pixels da costura, steam, e remove-os da imagem atual.

3.4.2 Apresente uma análise da complexidade desta operação em função da dimensão da imagem.

## 4. Validação

### 4.1 Crie uma função que receba uma imagem e um fator de escala como entrada, e que devolva a imagem redimensionada utilizando a API SeamCarving. A função deve ser capaz de lidar com o redimensionamento da largura e da altura de uma imagem, consoante a escolha do utilizador.

### 4.2 Reduza a largura da imagem img-broadway_tower.jpg para 70% da sua largura original, utilizando a função implementada. O resultado deve ser apresentado no próprio notebook.

### 4.3 Reduza a largura da imagem img-brent-cox-unsplash.jpg para 60% da sua altura original, utilizando a função implementada. O resultado deve ser apresentado no próprio notebook.

## 5. Questões Éticas

### a) Se colaborou com alguém fora do seu grupo, indique aqui os respetivos nomes.

Não.

### b) Deve citar todas as fontes utilizadas fora do material da UC.